In [1]:
import gzip

import geopandas as gpd
import ipywidgets as widgets
import numpy as np
import requests
from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_continuous_cmap
from matplotlib import colormaps
from matplotlib.colors import LogNorm

# Population data

Using [kontur population data](https://data.humdata.org/dataset/kontur-population-dataset-3km).

The full 3km dataset is ~2 million polygons. Unfortunately my machine can only handle ~1-1.5 million so we'll subset to Europe and North/South America.

In [2]:
continents = gpd.read_file(
    "https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/World_Continents/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson",
    use_arrow=True,
)

poi = ["Europe", "North America", "South America", "Africa"]

bbox = tuple(continents[continents.CONTINENT.isin(poi)].to_crs(3857).total_bounds)

poly_mask = (
    continents[continents.CONTINENT.isin(poi)]
    .to_crs(3857)
    .buffer(10000)
    .to_crs(4326)
    .union_all()
)

Read 3km grid data from URL.

In [3]:
url = "https://geodata-eu-central-1-kontur-public.s3.eu-central-1.amazonaws.com/kontur_datasets/kontur_population_20231101_r6.gpkg.gz"

with requests.get(url, stream=True, timeout=30) as res:
    pop_data = gzip.decompress(res.content)

Read data to `GeodataFrame` - ~2million polygons.

*note*: assumes the default I/O engine is `pyogrio` per [GeoPandas >=1.*](https://github.com/geopandas/geopandas/releases/tag/v1.0.0).

In [4]:
pop_gdf = gpd.read_file(
    pop_data,
    use_arrow=True,
).to_crs(4326)

len(pop_gdf)

/Users/jordanpinder/.pyenv/versions/3.10.13/lib/python3.10/contextlib.py:135: RuntimeWarning: File /vsimem/3e2836270c3f487b99df631fcaadcfde has GPKG application_id, but non conformant file extension
  return next(self.gen)


2016971

Spatially filter to relevant area - ~1.28 million polygons.

In [5]:
pop_gdf = pop_gdf.iloc[
    pop_gdf.sindex.query(
        poly_mask,
        predicate="intersects",
    )
]

len(pop_gdf)

1283241

In [6]:
heights = pop_gdf["population"].to_numpy()
heights = np.nan_to_num(heights, nan=1)

normalizer = LogNorm(1, heights.max(), clip=True)
normalized_heights = normalizer(heights)

colors = apply_continuous_cmap(normalized_heights, colormaps["plasma"])

pop_layer = PolygonLayer.from_geopandas(
    gdf=pop_gdf[["population", "geometry"]],
    extruded=True,
    get_elevation=heights,
    get_fill_color=colors,
)

In [7]:
view_state = {
    "longitude": -5,
    "latitude": 30,
    "zoom": 2,
    "pitch": 45,
    "bearing": 0,
}

map_layout = widgets.Layout(height="1200px")

m = Map(pop_layer, view_state=view_state, layout=map_layout)

m

Map(layers=[PolygonLayer(extruded=True, get_elevation=<pyarrow.lib.FloatArray object at 0x3d8ac4100>
[
  5270,…

In [9]:
m.to_html("examples/population.html")